In [1]:

import sys, os
cdir = %pwd
sys.path.append(os.path.join(cdir, '..', 'python'))
sys.path.append(os.path.join(cdir, 'kunst', 'python'))

In [2]:
from hue_show import *

In [3]:
!curl https://www.meethue.com/api/nupnp

[{"id":"001788fffe273a83","internalipaddress":"10.0.1.18"}]

In [4]:
from time import sleep
from phue import Bridge
import random
from numpy import zeros, ones
import numpy as np
import matplotlib.pyplot as plt
from tqdm import *
import requests as req
import pandas as pd
from rgb_cie import Converter

In [5]:
#bridges = req.get('https://www.meethue.com/api/nupnp').json()

In [6]:
bridges = [
    {'ip':'10.0.1.18', 'username':'sVzdqoRQzArjcl9Uw1-IXEEQUVCBsnQeVy-d4J0o'},
    {'ip':'81.161.138.239:6675', 'username':'80185a5af6f2a719125d712fc76dd3'},    
    {'ip':'81.161.138.239:6676', 'username':'EDTtC-PA4mzs7vVjrR0VWRaH3P4yYRqUIa2Ws6yP'},        
]

In [7]:
bri = []
for br in bridges:
    b = Bridge(**br)
    b.connect
    bri.append(b)

In [8]:
ln = 1
speed = 20 # m/s
distance = 7 #m between each lamp
time = 7 / (speed * 1E3 / (60*60))

def light(br, name, r,g,b):
    l = br.lights_by_name[name]
    rgbl(l, 1.0,0.0,0.0, brightness=254)

#normal()    
for n in range(9):
    try:
        name = "P%d"%((ln+n)%(19)+1)
        l = b.lights_by_name[name]
        rgbl(l, 1.0,0.0,0.0, brightness=254)
        sleep(time)
        rgbl(l, 1.,1.,1., brightness=254)
        sleep(0.1)
    except Exception as e:
        print(e)


'P2'
'P3'
'P4'
'P5'
'P6'
'P7'
'P8'
'P9'
'P10'


### Search for lights

In [9]:
import json
import time

In [10]:
fbri = lambda i, str_: 'http://{ip}/api/{username}/{str_}'.format(str_=str_, **bridges[i])
get = lambda i, str_:req.get(fbri(i, str_)).json()
post = lambda i, str_, data:req.post(fbri(i, str_), json.dumps(data)).json()
put = lambda i, str_,  data:req.put(fbri(i, str_), json.dumps(data)).json()

Search for new lights

In [28]:
post(1,'lights',{})

ConnectionError: ('Connection aborted.', ConnectionRefusedError(111, 'Connection refused'))

In [44]:
req.get('http://pe-desktop.local:4566/api/EDTtC-PA4mzs7vVjrR0VWRaH3P4yYRqUIa2Ws6yP/lights/1')

ConnectionError: ('Connection aborted.', ConnectionRefusedError(111, 'Connection refused'))

In [39]:
fbri(2, 'lights/1')

'http://localhost:4566/api/EDTtC-PA4mzs7vVjrR0VWRaH3P4yYRqUIa2Ws6yP/lights/1'

In [64]:
for i in range(len(bridges)):
    post(i,'lights',{})

In [65]:
print(get(0,'lights/new'))
print(get(1,'lights/new'))
print(get(2,'lights/new'))

{'lastscan': 'active'}
{'lastscan': 'active'}
{'lastscan': 'active'}


In [67]:
for i in range(len(bridges)):
    print('bridge-%d, now there are %d lights'%(i, len(get(i,'lights'))))

bridge-0, now there are 38 lights
bridge-1, now there are 12 lights
bridge-2, now there are 33 lights


In [213]:
#df_lamps = []

def update_state():
    lamps = {}
    for i in range(len(bridges)):
        #print('Bridge-%d'%(i))
        for k,v in get(i,'lights').items():
            v.update(v['state'])
            v['bridge'] = i
            v['relid'] = k
            #df_lamps.append(v)
            #if 'Hue color' in v['name']:
            #print(v['name'], v['state']['reachable'], v['uniqueid'])
            idl = v['uniqueid']
            if idl in lamps:
                lamps[idl][i] = v
            else:
                lamps[idl] = {i:v}
        time.sleep(0.1)
    return lamps

In [214]:
update_state()

{'00:17:88:01:00:b1:f8:dc-0b': {0: {'alert': 'none',
   'bri': 254,
   'bridge': 0,
   'colormode': 'hs',
   'ct': 500,
   'effect': 'none',
   'hue': 5000,
   'manufacturername': 'Philips',
   'modelid': 'LCT001',
   'name': 'Hue color lamp 17',
   'on': True,
   'reachable': False,
   'relid': '42',
   'sat': 254,
   'state': {'alert': 'none',
    'bri': 254,
    'colormode': 'hs',
    'ct': 500,
    'effect': 'none',
    'hue': 5000,
    'on': True,
    'reachable': False,
    'sat': 254,
    'xy': [0.675, 0.322]},
   'swversion': '5.23.1.13452',
   'type': 'Extended color light',
   'uniqueid': '00:17:88:01:00:b1:f8:dc-0b',
   'xy': [0.675, 0.322]}},
 '00:17:88:01:00:bf:3a:24-0b': {0: {'alert': 'none',
   'bri': 254,
   'bridge': 0,
   'colormode': 'hs',
   'ct': 500,
   'effect': 'none',
   'hue': 5000,
   'manufacturername': 'Philips',
   'modelid': 'LCT001',
   'name': 'Hue color lamp 16',
   'on': True,
   'reachable': False,
   'relid': '41',
   'sat': 254,
   'state': {'alert

In [170]:
df2.size

120

In [228]:
lamps = update_state()
for k,v in lamps.items():
    print(k,'-----')
    for l,m in v.items():
        print(l, m['state']['reachable'], m['name'], m['bridge'], m['relid'])
        if m['reachable']:
            put(m['bridge'], 'lights/%s/state'%(m['relid']), {"on":True, "sat":1, "bri":254,"hue":5000})
            time.sleep(0.5)
            print(get(m['bridge'], 'lights/%s'%(m['relid'])))
            time.sleep(0.5)
            

00:17:88:01:00:f5:71:01-0b -----
0 False P12 0 6
1 True Hue color lamp 8 1 11
{'type': 'Extended color light', 'name': 'Hue color lamp 8', 'modelid': 'LCT001', 'manufacturername': 'Philips', 'swversion': '66013187', 'state': {'xy': [0.0, 0.0], 'colormode': 'hs', 'alert': 'none', 'effect': 'none', 'reachable': True, 'sat': 1, 'ct': 0, 'hue': 5000, 'on': True, 'bri': 254}, 'uniqueid': '00:17:88:01:00:f5:71:01-0b'}
2 True Hue color lamp 6 2 47
{'type': 'Extended color light', 'name': 'Hue color lamp 6', 'modelid': 'LCT001', 'manufacturername': 'Philips', 'swversion': '5.23.1.13187', 'state': {'xy': [0.4596, 0.4105], 'colormode': 'hs', 'alert': 'none', 'effect': 'none', 'reachable': True, 'sat': 1, 'ct': 369, 'hue': 5000, 'on': True, 'bri': 254}, 'uniqueid': '00:17:88:01:00:f5:71:01-0b'}
00:17:88:01:00:dc:06:c9-0b -----
0 False Hue color lamp 12 0 36
2 False a3 2 15
00:17:88:01:00:ee:c7:5f-0b -----
0 False P5 0 10
2 True Hue color lamp 14 2 55
{'type': 'Extended color light', 'name': 'Hue 

In [53]:
class SvalinMetaBridge(object):
    default_bridges = [
        {'ip':'10.0.1.18', 'username':'sVzdqoRQzArjcl9Uw1-IXEEQUVCBsnQeVy-d4J0o'},
        {'ip':'81.161.138.239:6675', 'username':'80185a5af6f2a719125d712fc76dd3'},    
        {'ip':'81.161.138.239:6676', 'username':'EDTtC-PA4mzs7vVjrR0VWRaH3P4yYRqUIa2Ws6yP'},        
    ]
    
    all_lights_uid = ["DF1430", "7AC331", "B055E9", "1913F7", "2D2DA6", "0F9E4E", "DDE088", 
                      "209C5A", "E2B400", "6878AA", "34DE81", "62F1E6", "D9953F", "2A75A7", 
                      "5D3F46", "132A13", "81D5FE", "CF5C37"]
    
    def __init__(self, bridges=[]):
        if len(bridges) > 0:
            self.bridges = bridges
        else:
            self.bridges = self.default_bridges
        self.lamps = {}
        self.bridge_state = [{} for b in self.bridges]
        
    def _fbri(self, i, str_): 
        return 'http://{ip}/api/{username}/{str_}'.format(str_=str_, **self.bridges[i])
    
    def search_all(self):
        for i in range(len(self.bridges)):
            for j in self.all_lights_uid:
                post(i,'lights', {"deviceid":[j]})
                time.sleep(0.5)

    def _get(self, i, str_):
        return req.get(self._fbri(i, str_)).json()

    def _post(self, i, str_, data):
        return req.post(self._fbri(i, str_), json.dumps(data)).json()
    
    def _put(self, i, str_,  data):
        return req.put(self._fbri(i, str_), json.dumps(data)).json()
        
    def get_bridge_state(self, i):
        self.bridge_state[i] = self._get(i,'lights')
        return self.bridge_state[i]
    
    def all_ellipse(self):
        return (('P%d'%(i+1), self.find_uid('P%d'%(i+1))) for i in range(18))
        
    def update_state(self):
        for i in range(len(self.bridges)):
            for k,v in self.get_bridge_state(i).items():
                v.update(v['state'])
                v['bridge'] = i
                v['relid'] = k
                idl = v['uniqueid']
                if idl in self.lamps:
                    self.lamps[idl][i] = v
                else:
                    self.lamps[idl] = {i:v}
            time.sleep(0.01)
            
    def equal_state(self, uid, target, count=10):
        self.update_state()
        try:
            current=self.get_state(uid)
        except Exception as e:
            if 'light unreachable' in str(e):
                if count>0:                
                    time.sleep(0.5)
                    self.equal_state(uid, target, count=count-1)
                else:
                    raise e
        for k,v in target.items():
            if not target[k] == current[k]:
                return False
        return True

    def set_uid_state(self, uid, state, count=10, check=True):
        for k, v in self.lamps[uid].items():
            #if v['reachable']:
            self._put(k, 'lights/%s/state'%(v['relid']), state)
            #time.sleep(0.1)
        if check:
            if not self.equal_state(uid, state, count):
                if count>0:
                    time.sleep(1.0)
                    self.set_uid_state(uid, state, count=count-1)
                else:
                    raise Exception('Cant set state', uid, state, self.get_state(uid))
                
    def set_state(self, name, state, count=10, check=True):
        self.set_uid_state(self.find_uid(name), state, count, check)
        
    def set_RBG(self, name, R, G, B, count=10, check=True):
        self.set_state(name, {'xy': c.rgbToCIE1931(R, G, B), 'on': True, 'bri':254}, count, check)

    def get_state(self, uid):
        for k, v in self.lamps[uid].items():
            if v['reachable']:
                return v
        raise Exception('light unreachable', uid)
    
    def query_state(self, uid):
        for k, v in self.lamps[uid].items():
            if v['reachable']:
                v.update(self._get(k, 'lights/%d'%(v['relid'])))
                return v        
        
    def find_uid(self, name):
        for k,v in self.lamps.items():
            for b, w in v.items():
                if w['name'] == name:
                    return k
        raise Exception('cant find lamp named', name)
        
    def __getitem__(self, key):
        return pd.DataFrame(self.invert(self.lamps[self.find_uid(key)]))
    
    def __getattr__(self, key):
        return pd.DataFrame(self.invert(self.lamps[self.find_uid(key)]))    
        
    def invert(self, lamp):
        new_dic = {}
        for k,v in lamp.items():
            for l, w in v.items():
                new_dic[l] = [None for b in self.bridges]
        for b in range(len(self.bridges)):
            for w in new_dic.keys():
                if b in lamp:
                    new_dic[w][b]=lamp[b][w]
        return new_dic
    
    def get_attribute(self, key):
        for k,v in self.lamps.items():
            print(k,':')
            print(', '.join(['%d:%s'%(k2, str(w[key])) for k2, w in v.items()]))
                
        #return pd.DataFrame([(k, k2, w[key]) for k2, w in v.items() for k,v in self.lamps.items()], columns=['uid', 'bridge', key]
        #                   ).pivot(index='uid', columns='bridge', values=key)

In [54]:
s = SvalinMetaBridge()

In [55]:
s.update_state()

In [56]:
s.get_bridge_state(1)

{'10': {'manufacturername': 'Philips',
  'modelid': 'LCT001',
  'name': 'Hue color lamp 7',
  'state': {'alert': 'none',
   'bri': 0,
   'colormode': 'hs',
   'ct': 0,
   'effect': 'none',
   'hue': 0,
   'on': False,
   'reachable': False,
   'sat': 0,
   'xy': [0.0, 0.0]},
  'swversion': '66013187',
  'type': 'Extended color light',
  'uniqueid': '00:17:88:01:00:db:ad:de-0b'},
 '11': {'manufacturername': 'Philips',
  'modelid': 'LCT001',
  'name': 'Hue color lamp 8',
  'state': {'alert': 'none',
   'bri': 254,
   'colormode': 'ct',
   'ct': 369,
   'effect': 'none',
   'hue': 14910,
   'on': True,
   'reachable': True,
   'sat': 144,
   'xy': [0.4596, 0.4105]},
  'swversion': '66013187',
  'type': 'Extended color light',
  'uniqueid': '00:17:88:01:00:f5:71:01-0b'},
 '12': {'manufacturername': 'Philips',
  'modelid': 'LCT001',
  'name': 'Hue color lamp 9',
  'state': {'alert': 'none',
   'bri': 0,
   'colormode': 'hs',
   'ct': 0,
   'effect': 'none',
   'hue': 0,
   'on': False,
   '

In [57]:
s.query_state('00:17:88:01:00:ec:91:5b-0b')

In [72]:
s.update_state()
for name, uid in s.all_ellipse():
    print(uid, s.lamps[uid][0]['name'], s.invert(s.lamps[uid])['reachable'], any(s.invert(s.lamps[uid])['reachable']))
    s.set_RBG(name, 1.0, 1.0, 1.0, check=False)

00:17:88:01:00:d9:c9:cf-0b P1 [False, False, False] False
00:17:88:01:00:f5:8e:ff-0b P2 [True, None, False] True
00:17:88:01:00:d9:cb:67-0b P3 [True, False, False] True
00:17:88:01:00:ec:91:5b-0b P4 [False, None, None] False
00:17:88:01:00:ee:c7:5f-0b P5 [True, None, True] True
00:17:88:01:00:ec:91:a2-0b P6 [False, None, None] False
00:17:88:01:00:ee:c6:91-0b P7 [False, None, None] False
00:17:88:01:00:ee:c4:2e-0b P8 [False, True, False] True
00:17:88:01:00:ec:92:7e-0b P9 [False, None, False] False
00:17:88:01:00:d9:cb:35-0b P10 [False, False, False] False
00:17:88:01:00:f5:92:2e-0b P11 [False, True, False] True
00:17:88:01:00:f5:71:01-0b P12 [False, True, False] True
00:17:88:01:00:f5:7c:ae-0b P13 [False, True, False] True
00:17:88:01:00:f5:80:7f-0b P14 [False, False, False] False
00:17:88:01:00:dc:58:1b-0b P15 [True, False, False] True
00:17:88:01:00:dc:ea:dc-0b P16 [True, False, False] True
00:17:88:01:00:d9:bd:12-0b P17 [False, False, False] False
00:17:88:01:00:db:ad:de-0b P18 [Tr

In [498]:
s.P1.reachable

0    False
1     True
2    False
Name: reachable, dtype: bool

In [493]:
s.P4

,alert,bri,bridge,colormode,ct,effect,hue,manufacturername,modelid,name,on,reachable,relid,sat,state,swversion,type,uniqueid,xy
0,none,254,0,xy,369,none,14922,Philips,LCT001,P4,True,False,20,144,"{'xy': [0.167, 0.04], 'colormode': 'xy', 'aler...",5.5.1.9663,Extended color light,00:17:88:01:00:ec:91:5b-0b,"[0.167, 0.04]"
1,None,NaN,NaN,None,NaN,None,NaN,None,None,None,None,None,None,NaN,None,None,None,None,None
2,None,NaN,NaN,None,NaN,None,NaN,None,None,None,None,None,None,NaN,None,None,None,None,None


In [494]:
s.set_RBG('P4', 0., 0., 1.)

Exception: ('light unreachable', '00:17:88:01:00:ec:91:5b-0b')

In [476]:
s.P8

,alert,bri,bridge,colormode,ct,effect,hue,manufacturername,modelid,name,on,reachable,relid,sat,state,swversion,type,uniqueid,xy
0,none,254,0,hs,0,none,0,Philips,LCT001,P8,True,False,8,0,"{'xy': [0.0, 0.0], 'colormode': 'hs', 'alert':...",5.5.1.9663,Extended color light,00:17:88:01:00:ee:c4:2e-0b,"[0.0, 0.0]"
1,none,254,1,ct,369,none,14922,Philips,LCT001,Hue color lamp 1,True,True,4,144,"{'xy': [0.4595, 0.4105], 'colormode': 'ct', 'a...",66009663,Extended color light,00:17:88:01:00:ee:c4:2e-0b,"[0.4595, 0.4105]"
2,none,254,2,ct,219,none,0,Philips,LCT001,Hue color lamp 12,True,False,53,0,"{'xy': [0.1686, 0.0422], 'colormode': 'ct', 'a...",5.5.1.9663,Extended color light,00:17:88:01:00:ee:c4:2e-0b,"[0.1686, 0.0422]"


In [417]:
s._get(0, 'lights/23')

{'manufacturername': 'Philips',
 'modelid': 'LCT001',
 'name': 'P16',
 'state': {'alert': 'none',
  'bri': 254,
  'colormode': 'hs',
  'ct': 379,
  'effect': 'none',
  'hue': 14910,
  'on': True,
  'reachable': True,
  'sat': 254,
  'xy': [0.4656, 0.413]},
 'swversion': '5.23.1.13187',
 'type': 'Extended color light',
 'uniqueid': '00:17:88:01:00:dc:ea:dc-0b'}

In [421]:
s.set_state('P2', {'on':True, 'sat':254})

In [419]:
s.P1

,alert,bri,bridge,colormode,ct,effect,hue,manufacturername,modelid,name,on,reachable,relid,sat,state,swversion,type,uniqueid,xy
0,none,254,0,hs,0,none,0,Philips,LCT001,P1,True,False,22,254,"{'xy': [0.0, 0.0], 'colormode': 'hs', 'alert':...",5.23.1.13187,Extended color light,00:17:88:01:00:d9:c9:cf-0b,"[0.0, 0.0]"
1,none,254,1,hs,484,none,14910,Philips,LCT001,Hue color lamp 10,True,True,13,254,"{'xy': [0.52, 0.4362], 'colormode': 'hs', 'ale...",66013187,Extended color light,00:17:88:01:00:d9:c9:cf-0b,"[0.52, 0.4362]"
2,none,254,2,hs,364,none,0,Philips,LCT001,Hue color lamp 5,True,False,46,254,"{'xy': [0.2933, 0.2895], 'colormode': 'hs', 'a...",5.23.1.13187,Extended color light,00:17:88:01:00:d9:c9:cf-0b,"[0.2933, 0.2895]"


In [420]:
s.update_state()
s.P1.reachable

0    False
1     True
2    False
Name: reachable, dtype: bool

In [362]:
anew_dic).T

SyntaxError: invalid syntax (<ipython-input-362-ecbba7370044>, line 1)

In [278]:
s.set_state('00:17:88:01:00:f5:71:01-0b',  {"on":True, "sat":254, "bri":254,"hue":5000})

Exception: ('light unreachable', '00:17:88:01:00:f5:71:01-0b')

In [284]:
s.get_attribute('name')

00:17:88:01:00:f5:71:01-0b :
0:P12, 1:Hue color lamp 8, 2:Hue color lamp 6
00:17:88:01:00:dc:06:c9-0b :
0:Hue color lamp 12, 2:a3
00:17:88:01:00:ee:c7:5f-0b :
0:P5, 2:Hue color lamp 14
00:17:88:01:00:f5:80:7f-0b :
0:P14, 1:Hue color lamp 5, 2:Hue color lamp 8
00:17:88:01:00:d9:cc:a3-0b :
0:Hue color lamp 3, 2:k3
00:17:88:01:00:f5:7c:ae-0b :
0:P13, 1:Hue color lamp 3, 2:Hue color lamp 2
00:17:88:01:00:ee:c6:91-0b :
0:P7
00:17:88:01:00:f5:8e:ff-0b :
0:P2, 2:Hue color lamp 7
00:17:88:01:00:dc:58:1b-0b :
0:P15, 1:Hue color lamp 4, 2:Hue color lamp 3
00:17:88:01:00:ec:91:5b-0b :
0:P4
00:17:88:01:00:ec:95:ff-0b :
2:c4
00:17:88:01:00:b1:f8:dc-0b :
0:Hue color lamp 17
00:17:88:01:00:f5:90:b4-0b :
0:Hue color lamp 6, 2:b1
00:17:88:01:00:dc:fe:12-0b :
0:Hue color lamp 4, 2:a2
00:17:88:01:00:ec:91:a2-0b :
0:P6
00:17:88:01:00:db:ad:de-0b :
0:P18, 1:Hue color lamp 7, 2:Hue color lamp 16
00:17:88:01:00:ff:ec:16-0b :
0:AJ
00:17:88:01:00:f5:4a:93-0b :
0:Hue color lamp 7, 2:b3
00:17:88:01:00:dc:ea:dc-0

In [218]:
all_lights_uid = ["DF1430", "7AC331", "B055E9", "1913F7", "2D2DA6", "0F9E4E", "DDE088", "209C5A", "E2B400", "6878AA", "34DE81", "62F1E6", "D9953F", "2A75A7", "5D3F46", "132A13", "81D5FE", "CF5C37"]

In [219]:
len(all_lights[0:10])

NameError: name 'all_lights' is not defined

In [220]:
import json

In [221]:
import time

In [222]:
for i in range(len(bri)):
    post(i,'lights', {"deviceid":all_lights_uid[0:10]})
    time.sleep(1)
    post(i,'lights', {"deviceid":all_lights_uid[10:]})
    time.sleep(1)
    

### PARTY MODE

In [66]:
for i in range(len(bri)):
    put(i, 'groups/0/action', 
        {"on": True,
         "hue": 2000,
        })
    time.sleep(1)

In [ ]:
class Controler(object):
    def __init__(self):
        self.targets = {}
        

In [165]:
import pandas as pd

In [166]:
all_lamps = pd.DataFrame(df_lamps)

In [167]:
uid = all_lamps.groupby('uniqueid')

In [168]:
df2 = pd.DataFrame(uid.apply(lambda x: any(x['reachable'])))
df2['bridge'] = uid.apply(lambda x: [x['bridge']])
df2['names'] = uid.apply(lambda x: [x['name']])


In [169]:
df2

,0,bridge,names
uniqueid,,,
00:17:88:01:00:b1:f8:dc-0b,True,[[0]],[[Hue color lamp 17]]
00:17:88:01:00:bf:3a:24-0b,True,[[0]],[[Hue color lamp 16]]
00:17:88:01:00:cf:a5:dc-0b,False,[[0]],[[Hue lightstrip 1]]
00:17:88:01:00:d9:bd:12-0b,True,[[0]],[[P17]]
00:17:88:01:00:d9:c9:cf-0b,False,[[0]],[[P1]]
00:17:88:01:00:d9:ca:66-0b,True,[[0]],[[P19]]
00:17:88:01:00:d9:cb:35-0b,True,[[0]],[[P10]]
00:17:88:01:00:d9:cb:67-0b,True,[[0]],[[P3]]
00:17:88:01:00:d9:cc:a3-0b,True,[[0]],[[Hue color lamp 3]]
